# APPROACH 1 Code :  Optimizing Random Forest Model using Hypertuning and selecting the best model based on the lowest cross-validated RMSE

### Step 1: Data Preparation

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.linear_model import Lasso, Ridge , LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor , VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score, GridSearchCV

##### 1.1 Load and prepare the dataset

In [3]:
data = pd.read_csv("data.csv")

In [4]:
# Replace infinite values with a large number (e.g., 1e10)
data['AI_Workload_Ratio'].replace([np.inf, -np.inf], 1e10, inplace=True)

In [5]:
missing_values = data.isnull().sum()
print(missing_values)

Job titles           0
AI Impact            0
Tasks                0
AI models            0
AI_Workload_Ratio    0
Domain               0
dtype: int64


In [6]:
infinite_values = data[data == np.inf].sum()
print(infinite_values)

Job titles             0
AI Impact              0
Tasks                0.0
AI models            0.0
AI_Workload_Ratio    0.0
Domain                 0
dtype: object


In [7]:
data['AI Impact'] = data['AI Impact'].str.rstrip('%').astype(float)/100

##### 1.2 Handle missing values (if any)

In [8]:
missing_values = data.isnull().sum()

inf_values = (data['AI_Workload_Ratio'] == float('inf')).sum()

missing_values, inf_values

(Job titles           0
 AI Impact            0
 Tasks                0
 AI models            0
 AI_Workload_Ratio    0
 Domain               0
 dtype: int64,
 0)

##### 1.3 Encode categorical variables


In [9]:
data = pd.get_dummies(data, columns=["Job titles", "Domain"], drop_first=True)

In [10]:
data.columns

Index(['AI Impact', 'Tasks', 'AI models', 'AI_Workload_Ratio',
       'Job titles_3D Animator', 'Job titles_3D Artist',
       'Job titles_3D Designer', 'Job titles_3D Modeler',
       'Job titles_911 Dispatcher', 'Job titles_911 Operator',
       ...
       'Job titles_Zoologist', 'Domain_Communication & PR',
       'Domain_Construction', 'Domain_Data & IT', 'Domain_Hospitality',
       'Domain_Law Enforcement', 'Domain_Leadership & Strategy',
       'Domain_Medical & Healthcare', 'Domain_Sales & Marketing',
       'Domain_Supply Chain & Logistics'],
      dtype='object', length=4718)

##### 1.4 Split the data into features (X) and target (y)

In [11]:
X = data.drop(columns=["AI Impact"])
y = data["AI Impact"]

##### 1.5 Split the data into training and testing sets

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### 1.6 Choose a regression model and train it

In [13]:
models = [
    ("Linear Regression", LinearRegression()),
    ("Random Forest Regression", RandomForestRegressor()),
    ("Gradient Boosting Regression", GradientBoostingRegressor()),
    ("Lasso Regression", Lasso(alpha=0.01)),  
    ("Ridge Regression", Ridge(alpha=0.01)), 
]

##### 1.7 Training and evaluating multiple models 

In [14]:
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r_squared = r2_score(y_test, y_pred)
    
    print(f"Model: {name}")
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'R-squared: {r_squared}')
    print("--------")

Model: Linear Regression
MSE: 0.039977242787945205
RMSE: 0.1999430988755181
R-squared: -0.10282736474255905
--------
Model: Random Forest Regression
MSE: 0.02088878767515924
RMSE: 0.14452953910934346
R-squared: 0.42375398957205623
--------
Model: Gradient Boosting Regression
MSE: 0.021083420924632452
RMSE: 0.145201311718016
R-squared: 0.4183847630161781
--------
Model: Lasso Regression
MSE: 0.029024849945861398
RMSE: 0.17036680998909792
R-squared: 0.1993094934627484
--------


c:\Users\Anubhav Chachra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


Model: Ridge Regression
MSE: 0.029027314147090415
RMSE: 0.17037404188165053
R-squared: 0.1992415150740935
--------


In [15]:
best_model = None
best_rmse = float('inf')

for name, model in models:
    if name == "Random Forest Regression":
        # Feature Scaling (for Random Forest only)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Feature Selection (for Random Forest only)
        select = SelectFromModel(RandomForestRegressor(n_estimators=100))
        select.fit(X_train_scaled, y_train)
        X_train_selected = select.transform(X_train_scaled)
        X_test_selected = select.transform(X_test_scaled)
        
        # Hyperparameter tuning for Random Forest
        param_grid = {'n_estimators': [50, 100, 200]}
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X_train_selected, y_train)
        best_model = grid_search.best_estimator_
        
    else:
        # For other models
        model.fit(X_train, y_train)
        
    # Cross-validation for model selection
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    mse_cv = -np.mean(cv_scores)
    rmse_cv = np.sqrt(mse_cv)
    
    print(f"Model: {name}")
    print(f'RMSE on CV: {rmse_cv}')
    print("--------")
    
    if rmse_cv < best_rmse:
        best_rmse = rmse_cv
        best_model = model

# Step 6: Model Comparison
print(f"The best model is {best_model} with RMSE: {best_rmse}")

Model: Linear Regression
RMSE on CV: 0.18426652744271962
--------
Model: Random Forest Regression
RMSE on CV: 0.14180028160860778
--------
Model: Gradient Boosting Regression
RMSE on CV: 0.1401888048746886
--------
Model: Lasso Regression
RMSE on CV: 0.16328585865295198
--------


c:\Users\Anubhav Chachra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Users\Anubhav Chachra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Users\Anubhav Chachra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Users\Anubhav Chachra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
c:\Users\Anubhav Chachra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklea

Model: Ridge Regression
RMSE on CV: 0.16346154446720634
--------
The best model is GradientBoostingRegressor() with RMSE: 0.1401888048746886
